# Ajout des temps de rétention prédits à la base de données NORMAN-CCSbase

# Introduction
Ce notebook permet d'enrichir la base de données précédemment créée avec les temps de rétention (RT) prédits grâce au modèle QSRR disponible sur https://github.com/narvall018/QSRR-Retention-Time-Prediction.

# Import des bibliothèques et du modèle

In [ ]:
import pandas as pd
from QSRR_predictor import RTPredictor

# Chargement de la base de données existante

In [ ]:
# Ouverture du fichier HDF5
with pd.HDFStore("norman_all_ccs_pos_neg.h5") as store:
    keys = store.keys()
    print(f"Clés disponibles dans le fichier : {keys}")
    
    dfs = [store[key] for key in keys]
    df = pd.concat(dfs, ignore_index=True)


# Préparation des structures pour la prédiction

In [ ]:
# Extraction des SMILES uniques
smiles_uniques = df['SMILES'].unique()


# Prédiction des temps de rétention


In [ ]:
# Initialisation et prédiction
predictor = RTPredictor()
results = predictor.predict(smiles_uniques)


# Intégration des prédictions

In [ ]:
# Fusion des données
merged_df = pd.merge(df, 
                    results[['Original_SMILES', 'Predicted_RT']], 
                    left_on='SMILES', 
                    right_on='Original_SMILES', 
                    how='left')

# Nettoyage
merged_df.drop(columns=['Original_SMILES'], inplace=True)



# Finalisation et sauvegarde

In [ ]:
# Nettoyage final
final_df = merged_df.dropna(subset=['adduct'])

# Séparation des modes d'ionisation
final_positive = final_df[final_df['adduct'].str.endswith('+')]
final_negative = final_df[final_df['adduct'].str.endswith('-')]

print(f"Mode positif : {len(final_positive)} composés")
print(f"Mode négatif : {len(final_negative)} composés")

# Sauvegarde
final_positive.to_hdf("norman_all_ccs_all_rt_pos_neg.h5", key="positive")
final_negative.to_hdf("norman_all_ccs_all_rt_pos_neg.h5", key="negative")